In [33]:
import MDAnalysis
from MDAnalysis.analysis import distances
from MDAnalysis.analysis.hydrogenbonds.hbond_analysis import HydrogenBondAnalysis
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy
from scipy import stats

In [36]:
#############
#Run a bound water analysis for a set of structures
#Inputs:
#(a) pdb_path path to the pdb overall dir
#(b) list_pdbs list of pdb structures to work with
#(c) output_dir where to place output.npy
#(d) cutoff_d cutoff for bound water contact
#############
def run_for_bd_water(pdb_path, list_pdbs, output_dir, cutoff_d):
    
    #Keys- pdbs, values- resid lists and also average KD hydropathies
    w_info = {}
    
    #Iterate over each pdb
    for ipdb, pdb_analyze in enumerate(list_pdbs):
        
        print(f"On {pdb_analyze} index {ipdb}")
        
        #############
        #Download PDBBind files, make universes
        #############
        protein_file = f"{pdb_path}/{pdb_analyze}/{pdb_analyze}_protein_processed.pdb"
        u_prot = MDAnalysis.Universe(protein_file)
        
        #Copied below from https://github.com/gcorso/DiffDock/blob/main/datasets/pdbbind.py
        #Only work with mol2 though revise because that is compatible with MDAnalysis- sdf is not
        for file in os.listdir(os.path.join(pdb_path, pdb_analyze)):
            if file.endswith(".mol2") and 'rdkit' not in file:
                ligand_file = os.path.join(pdb_path, pdb_analyze, file)
                #if lig is None and os.path.exists(os.path.join(pdb_path, pdb_analyze, file[:-4] + ".mol2")):  # read mol2 file if sdf file cannot be sanitized
                #    print('Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.')
                #    lig = read_molecule(os.path.join(pdb_path, pdb_analyze, file[:-4] + ".mol2"), remove_hs=False, sanitize=True)
                
        
        #ligand_file = obtaif"{pdb_path}/5zxk/5zxk_ligand.mol2"
        
        u_ligand = MDAnalysis.Universe(ligand_file)
        
        #############
        #Protein and ligand info setup
        #############
        #For now- all segids allowed
        #May want to only have one later?
        psegids = list(set([a.segid for a in u_prot.atoms]))
        psegid_str = " ".join([p for p in psegids])
        #if len(psegids) > 1:
        #    print("over 1 protein segids")
        #    print(psegids)
        #psegid = psegids[0]
        #print(u_ligand.atoms)
        lresnames = list(set([a.resname for a in u_ligand.atoms]))
        if len(lresnames) > 1:
            print("over 1 ligand resnames")
            print(lresnames)
        lresname = lresnames[0]
        
        #############
        #Find nearby waters in pdb
        #############
        #Now pull from rcsb
        #Ref https://docs.mdanalysis.org/stable/documentation_pages/coordinates/MMTF.html
        #rcsb_entry = "5ZXK"
        u_rcsb = MDAnalysis.coordinates.MMTF.fetch_mmtf(pdb_analyze)
        
        u_rcsb_protein_sel = f"(protein and segid {psegid_str})"
        u_rcsb_ligand_sel = f"(resname {lresname})"
        p_near_l_sel = f"{u_rcsb_protein_sel} and around 3.3 {u_rcsb_ligand_sel}"
        l_near_p_sel = f"{u_rcsb_ligand_sel} and around 3.3 {u_rcsb_protein_sel}"
        #print("p_near_l_sel")
        #print(p_near_l_sel)
        #print("l_near_p_sel")
        #print(l_near_p_sel)
        
        #TODO confirm always water is named HOH
        water_atoms = u_rcsb.select_atoms("resname HOH")
        if len(water_atoms) < 1:
            print(f"NO WATERS PDB {pdb_analyze}")
        #print(f"resname HOH and around {cutoff_d} ((protein and segid {psegid_str}) and (resname {lresname}))")
        nearby_water_sel = u_rcsb.select_atoms(f"resname HOH and (around {cutoff_d} (protein and segid {psegid_str})) and (around {cutoff_d} (resname {lresname}))")
        #print(nearby_water_sel)
        
        unique_w_resids = list(set([r.resid for r in nearby_water_sel]))
        #print("unique resids")
        #print(unique_w_resids)
        
        #record
        w_info[pdb_analyze] = len(unique_w_resids)
        
        #############
        #H bond calc
        #############
        '''
        #Ref https://userguide.mdanalysis.org/stable/universe.html
        #Ref https://docs.mdanalysis.org/2.6.1/documentation_pages/core/universe.html#MDAnalysis.core.universe.Merge
        #Merge
        u_pl = MDAnalysis.core.universe.Merge(u_prot.atoms, u_ligand.atoms)
        
        #Run HB analysis
        #Ref https://docs.mdanalysis.org/stable/documentation_pages/analysis/hydrogenbonds.html
        #HB create
        plhbonds = MDAnalysis.analysis.hydrogenbonds.hbond_analysis.HydrogenBondAnalysis(
          universe=u_pl,
          between=["protein", f"resname {lresname}"]
          )

        #Protein and ligand selections
        protein_hydrogens_sel = plhbonds.guess_hydrogens("protein")
        protein_acceptors_sel = hbonds.guess_acceptors(u_rcsb_protein_sel)

        l_hydrogens_sel = hbonds.guess_hydrogens(f"resname {lresname}")
        l_acceptors_sel = hbonds.guess_acceptors(f"resname {lresname}")

        #Combine
        plhbonds.hydrogens_sel = f"({u_rcsb_protein_sel} and ({protein_hydrogens_sel})) or ({u_rcsb_ligand_sel} and ({l_hydrogens_sel}))"
        plhbonds.acceptors_sel = f"({u_rcsb_protein_sel} and ({protein_acceptors_sel})) or ({u_rcsb_ligand_sel} and ({l_acceptors_sel}))"

        print("H sel")
        print(plhbonds.hydrogens_sel)

        print("Acc sel")
        print(plhbonds.hydrogens_sel)

        plhbonds.run()
        print(plhbonds)
        '''
        
    #Save this info
    np.save(f"{output_dir}/Nearby_W_Cutoff_{cutoff_d}.npy", w_info)

In [37]:
pdb_dir = "/Users/dsharon/Documents/MIT/6.8701/Project/Code/HarmonicFlow/FlowSite/data/PDBBind_processed"
pdb_list = "/Users/dsharon/Documents/MIT/6.8701/Project/Data/From_Hannes/TEST3_top40_epoch75_FILTER_restart_cacheNewRestart_big_ema_ESM2emb_tr34_WITH_fixedSamples28_id1_FILTERFROM_temp_restart_ema_ESM2emb_tr34/complex_names.npy"
odir = "/Users/dsharon/Documents/MIT/6.8701/Project/Analysis/RMSD_and_Chem_Feats/Site_Comp_231128"
dcutoff = 3.70

#From Hannes
with open(pdb_list, 'rb') as f:
    complex_names = np.load(f)

run_for_bd_water(pdb_dir, complex_names, odir, dcutoff)

On 6qqw index 0
On 6d08 index 1
On 6jap index 2
On 6np2 index 3
On 6uvp index 4
On 6oxq index 5
On 6jsn index 6
On 6hzb index 7
On 6qrc index 8
On 6oio index 9
On 6jag index 10
On 6moa index 11
On 6hld index 12
On 6i9a index 13
On 6e4c index 14
On 6g24 index 15
On 6jb4 index 16
On 6s55 index 17
On 6seo index 18
On 6dyz index 19
On 5zk5 index 20
On 6jid index 21
On 5ze6 index 22
On 6qlu index 23
On 6a6k index 24
On 6qgf index 25
On 6e3z index 26
On 6te6 index 27
On 6pka index 28
On 6g2o index 29
On 6jsf index 30
On 5zxk index 31
On 6qxd index 32
On 6n97 index 33
On 6jt3 index 34
On 6qtr index 35
On 6oy1 index 36
On 6n96 index 37
On 6qzh index 38
On 6qqz index 39
On 6qmt index 40
On 6ibx index 41
On 6hmt index 42
On 5zk7 index 43
On 6k3l index 44
On 6cjs index 45
On 6n9l index 46
On 6ibz index 47
On 6ott index 48
NO WATERS PDB {pdb_analyze}
On 6gge index 49
On 6hot index 50
On 6e3p index 51
On 6md6 index 52
On 6hlb index 53
On 6fe5 index 54
On 6uwp index 55
On 6npp index 56
On 6g2f index

/Users/dsharon/anaconda3/envs/mda_env_2/lib/python3.11/site-packages/MDAnalysis/topology/MOL2Parser.py:234: UserWarning: Unknown elements found for some atoms: {'B'}. These have been given an empty element record.
  warnings.warn("Unknown elements found for some "
/Users/dsharon/anaconda3/envs/mda_env_2/lib/python3.11/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: 
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


On 6jsz index 223
On 6o9c index 224
On 6mo8 index 225
On 6qln index 226
On 6qqu index 227
On 6i66 index 228
On 6mja index 229
On 6gwe index 230
On 6d3z index 231
On 6oxr index 232
On 6r4k index 233
On 6hle index 234
On 6h9v index 235
On 6hou index 236
On 6nv9 index 237
On 6py0 index 238
On 6qlq index 239
On 6nv7 index 240
On 6n4b index 241
NO WATERS PDB {pdb_analyze}
On 6jaq index 242
On 6i8m index 243
On 6dz0 index 244
On 6oxs index 245
On 6k2n index 246
On 6cjj index 247
On 6ffg index 248
On 6a73 index 249
On 6qqt index 250
On 6a1c index 251
On 6oxu index 252
On 6qre index 253
On 6qtw index 254
On 6np4 index 255
On 6hv2 index 256
On 6n55 index 257
On 6e3o index 258
On 6kjd index 259
On 6sfc index 260
On 6qi7 index 261
On 6hzc index 262
On 6k04 index 263
On 6op0 index 264
On 6q38 index 265
On 6n8x index 266
On 6np3 index 267
On 6uvv index 268
On 6pgo index 269
On 6jbe index 270
On 6i75 index 271
On 6qqq index 272
On 6i62 index 273
On 6j9y index 274
On 6g29 index 275
On 6h7d index 276
